In [5]:
import torch
from transformers import BertTokenizer, BertModel
from torch import nn
import numpy as np

# BERT 모델 클래스 정의
class BERTClassifier(nn.Module):
    def __init__(self, bert_model, num_classes, dropout_rate=0.3):
        super(BERTClassifier, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)

        #  position_ids 추가
        self.bert.embeddings.register_buffer("position_ids", torch.arange(512).expand((1, -1)))
        
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

# 필요한 변수들 설정
model_name = "klue/bert-base"
tokenizer = BertTokenizer.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 6  # 감정 클래스의 수에 맞게 조정

# BERT 모델 로드
bert_model = BertModel.from_pretrained(model_name)

# 모델 초기화 및 가중치 로드
model = BERTClassifier(bert_model=bert_model, num_classes=num_classes)
model.load_state_dict(torch.load('bert_classifier_fold_dataset2_5.pth', map_location=device), strict=False)
model.to(device)
model.eval()

# 감정 레이블 (학습 시 사용한 레이블 순서와 일치해야 함)
emotion_labels = ['기쁨', '슬픔', '분노', '불안', '당황', '상처']

# 텍스트 감정 분류 함수
def classify_emotion(text):
    encoded_input = tokenizer(text, padding=True, truncation=True, max_length=128, return_tensors="pt")
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        predicted_class = torch.argmax(probabilities, dim=1).item()
    
    predicted_emotion = emotion_labels[predicted_class]
    confidence = probabilities[0][predicted_class].item()
    
    return predicted_emotion, confidence

# 테스트
test_texts = [
    "오늘은 정말 행복한 날이에요!",
    "이 상황이 너무 화가 나고 짜증나요.",
    "앞으로 어떻게 될지 너무 걱정돼요.",
    "아무 느낌이 들지 않아요.",
    "갑자기 일이 생겨서 놀랐어요.",
    "정말 깜짝 놀랐어",
    "밥은 먹었니?",
    "워우 정말 죽을뻔했다",
    "오늘 너무 힘들었어",
    "너 괜찮아?? 너 아파보여"
]

for text in test_texts:
    emotion, confidence = classify_emotion(text)
    print(f"텍스트: {text}")
    print(f"예측된 감정: {emotion}")
    print(f"신뢰도: {confidence:.4f}")
    print()

텍스트: 오늘은 정말 행복한 날이에요!
예측된 감정: 기쁨
신뢰도: 0.9995

텍스트: 이 상황이 너무 화가 나고 짜증나요.
예측된 감정: 분노
신뢰도: 0.9982

텍스트: 앞으로 어떻게 될지 너무 걱정돼요.
예측된 감정: 불안
신뢰도: 0.9486

텍스트: 아무 느낌이 들지 않아요.
예측된 감정: 기쁨
신뢰도: 0.5323

텍스트: 갑자기 일이 생겨서 놀랐어요.
예측된 감정: 당황
신뢰도: 0.7403

텍스트: 정말 깜짝 놀랐어
예측된 감정: 당황
신뢰도: 0.9656

텍스트: 밥은 먹었니?
예측된 감정: 기쁨
신뢰도: 0.9545

텍스트: 워우 정말 죽을뻔했다
예측된 감정: 상처
신뢰도: 0.6010

텍스트: 오늘 너무 힘들었어
예측된 감정: 당황
신뢰도: 0.9329

텍스트: 너 괜찮아?? 너 아파보여
예측된 감정: 상처
신뢰도: 0.4748

